The main question is how to calculate permutation test with repeated measures (RM) anova using different clusters (per-defined groups of channels). Similar to the MNE function: https://martinos.org/mne/stable/generated/mne.stats.permutation_cluster_test.html

In [6]:
import os, sys
import pandas as pd
import numpy as np
import mne
from gc import collect as clear
from eelbrain import *

try:
    if "winsound" not in sys.modules:
        import winsound
    def makeSound(freq = 6000, # Hz
              duration = 3000): # millisecond
        winsound.Beep(freq, duration)
except ImportError:
    if "os" not in sys.modules:
        import winsound
    def makeSound():
        os.system('say -v Amir''s Task finished!')

Importing sample data with pandas.

It's 60 files (~260kb per file). 3 groups, 10 couples, 2 repeated measures. 

To make it simple I used balanced design. My real data is unbalanced. 

1. The first two columns represents the channels' name
2. alpha/beta/gamma represents the PLV values for the pair of channels
3. id - couples' number
4. group - 1/2/3 the group condition. Between subjects. 
5. measure - m/f the first/second measure. Within subjects, my repeated measure. 

In [7]:
files = [x[0]+"/"+f for x in os.walk("data") for f in x[2] if
               f.endswith(".csv")] 

chnames = ['Fp1-0', 'Fp2-0', 'F7-0', 'F8-0', 'F3-0', 'F4-0', 'Fz-0', 'FT9-0', 'FT10-0', 'FC5-0', 'FC1-0',
 'FC2-0', 'FC6-0', 'T7-0', 'C3-0', 'Cz-0', 'C4-0', 'T8-0', 'TP9-0', 'CP5-0', 'CP1-0', 'CP2-0',
 'CP6-0', 'TP10-0',  'P7-0', 'P3-0', 'Pz-0', 'P4-0', 'P8-0', 'O1-0', 'O2-0', 'Fp1-1', 'Fp2-1',
 'F7-1', 'F8-1', 'F3-1', 'F4-1', 'Fz-1', 'FT9-1', 'FT10-1', 'FC5-1', 'FC1-1', 'FC2-1', 'FC6-1',
 'T7-1', 'C3-1', 'Cz-1', 'C4-1', 'T8-1', 'TP9-1', 'CP5-1', 'CP1-1', 'CP2-1',  'CP6-1',  'TP10-1',
 'P7-1',  'P3-1',  'Pz-1',  'P4-1',  'P8-1',  'O1-1', 'O2-1']


my shape is: (60, 6, 3844) (file, columns(channel_1, channel_2, alpha, id, group, measure), plv values)
[['Fp1-0' 'Fp1-0' 'Fp1-0' ... 'O2-1' 'O2-1' 'O2-1']
 ['Fp1-0' 'Fp2-0' 'F7-0' ... 'P8-1' 'O1-1' 'O2-1']
 [0.0 0.007969122912060941 -0.04311775644267104 ... 0.024559873633983688
  0.02824765472010002 0.0]
 [332.0 332.0 332.0 ... 332.0 332.0 332.0]
 ['st' 'st' 'st' ... 'st' 'st' 'st']
 ['f' 'f' 'f' ... 'f' 'f' 'f']]


In [28]:
files[0]

'data/N332_f_conn.csv'

In [22]:
pairs = [(ch, ch2) for i, ch in enumerate(chnames) for ch2 in chnames[i + 1:]]
labels = [' '.join(pair) for pair in pairs]

connectivity = []
for i, pair1 in enumerate(pairs):
    for pair2 in pairs[i + 1:]:
        ch1, ch2 = pair1
        if ch1 in pair2 or ch2 in pair2:
            connectivity.append((' '.join(pair1), ' '.join(pair2)))


Converting channel names from numeric to actual names, changing group names to string, and changing id type to float.

The suffix -0 or -1 reflects if it is the first or second participant. 

**Starting to use *eelbrain**.

Creating `ds()` object from my pandas Data.Frame. For string columns I used `factor`, for numeric columns I used `NDVar`. 

Allowed connectivities between sensors

In [24]:
pairs = set()
for src, dst in connectivity:
    a = labels.index(src)
    b = labels.index(dst)
    if a < b:
        pairs.add((a, b))
    else:
        pairs.add((b, a))
connectivity = np.array(sorted(pairs), np.uint32)
sensor_dim = Categorial("pair", labels, connectivity)

In [27]:
len(sensor_dim)

1891

In [42]:
rows = []
for file in files:
    items = file.split('/')[1].split('_')
    subject = items[0][1:]
    condition = items[1]
    group = items[0][1]
    
    df = pd.read_csv(file, usecols=['channel_1', 'channel_2', "alpha"], header=0)
    df[["channel_1","channel_2"]] = df[["channel_1","channel_2"]].replace(list(range(62)), chnames)
    d = {' '.join((ch1, ch2)): v for ch1, ch2, v in df.values}
    data = np.array([d[l] for l in labels])
    plv = NDVar(data, sensor_dim)
    rows.append([subject, condition, group, plv])

In [43]:
rows[0]

['332', 'f', '3', <NDVar: 1891 pair>]

In [45]:
ds = Dataset.from_caselist(['subject', 'condition', 'group', 'plv'], rows)

In [48]:
ds.summary()
ds['subject'].random = True

In [49]:
ds.head()

subject   condition   group
---------------------------
332       f           3    
226       f           2    
142       m           1    
237       m           2    
142       f           1    
226       m           2    
332       m           3    
237       f           2    
223       m           2    
139       f           1    

In [50]:
ds['plv']

<NDVar 'plv': 60 case, 1891 pair>

In [51]:
res = testnd.anova('plv', 'condition * group * subject(group)', ds=ds, pmin=0.05)

Permutation test: 100%|██████████| 10000/10000 [00:24<00:00, 415.37 permutations/s]


Some basic info about my dataset

In [52]:
res.table()

#   Effect              f_max      p   sig
------------------------------------------
0   condition           13.56   .309      
1   group               10.03   .190      
2   condition x group   11.69   .455      

In [58]:
print(res.find_clusters(0.4))

id   v        p        sig   effect   
--------------------------------------
1    556.99   0.3094         condition
1    543.16   0.1904         group    


In [57]:
res.cluster(1, 'group')

array([0., 0., 0., ..., 0., 0., 0.])

If I understood correctly, I should use the `info` attribute to create my clusters.

So, let's say I have three clusters for the following channels: 
1. F4 and F3
2. T7, CP5, and P7
3. CP6, T8, and P8

How can I add and use these clusters? If I want to see if F4*-0* (in my first participant) is related to F3*-1* (in my second participant)? or T7*-0* is related to T7*-1*?

Regular RM anova for the data. While it's a bit redundant, I wanted to see that I can run it.  

In [ ]:
print(test.anova(ds["alpha"], ds["group"] * ds["measure"]))

Next, I moved to permutation RM anova with clusters. 
1. I understand that I have some issues with the way I build the NDVar for `id`. I didn't understand how to fix it. 
2. How should I specify the cluster?  

In [ ]:
print(testnd.anova(ds["alpha"], ds["group"] * ds["measure"] * ds["id"], match=False, samples=100, title="RM with permutation:"))